In [1]:
import time
import glob,random
import datetime
import os
import subprocess
import shlex
import gc
import pandas as pd
import collections
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import ensemble, metrics 
import gc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from imblearn.over_sampling import (RandomOverSampler, SMOTE, ADASYN)
from collections import Counter
import sys, traceback
import threading
import datetime

In [2]:
disk_model_name = "ST4000DM000"
#disk_model_name = "ST8000DM002"
#disk_model_name = "ST8000NM0055"
#disk_model_name = "ST12000NM0007"
#disk_model_name = "ST6000DX000" //error
#disk_model_name = "ST10000NM0086" //error

number_of_days = 1

map_list = []
index_map ={}
date_dict={}
now = time.time()
count=-1
file_name=""

In [3]:
with open("../map_2019.txt","r")as in_file:
    for line in in_file:
        if(len(line.strip())>0):
            if".csv" in line:
                if(count!=-1):
                    map_list.append(date_dict)
                    index_map[file_name]=count
                
                date_dict.clear()
                count+=1
                file_name=line.strip()
            else:
                parts = line.strip().split(" ")
                date_dict[parts[0]] = int(float(parts[1]))
                    
#                print(line)
                
                
print(count)

89061


In [4]:
def get_lable(serial_number_list,date,year,month,day):
    global parent_folder_name
    next_day_label =[]
    
    now = datetime.datetime(year,month,day)
    next_day = (now + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    
    for x in serial_number_list:
        index = index_map[x+".csv"]
        current_value = map_list[index][date]
        
        next_day_value = map_list[index][next_day]
        
        if(current_value==next_day_value):
            next_day_label.append(0)
        else:
            next_day_label.append(1)
        
    return next_day_label

In [5]:
def calculate_error_prob(prob_list):
    final_prob =0
    for x in range(len(prob_list)):
        i = prob_list[x]
        for y in range(len(prob_list)):
            if x!=y:
                i=i*(1-prob_list[y])
        final_prob+=i
#    print(final_prob)
    return final_prob

In [6]:
def get_best_disk(prob_list, n, threshold):
    alpha = .9
    min_cost = 999999
    from itertools import combinations, chain
#    n = len(serial_number_list)
    allsubsets = lambda n: list(chain(*[combinations(range(n), ni) for ni in range(n+1)]))
    selected_subset=[]
    all_list=[i for i in range(n)]
    for x in allsubsets(n):
        temp_list=[]
        for y in x:
            temp_list.append(y)
        
        
        un_selected_list=(list(set(all_list) - set(temp_list)))
        slected_prob_list = []
        for i in un_selected_list:
            slected_prob_list.append(prob_list[:,1][i])
        
        prob = calculate_error_prob(slected_prob_list)
        if(prob<=threshold):
            data_size = len(x)/n
            cost = (alpha*data_size)+(1-alpha)*prob
            if(cost<min_cost):
                min_cost = cost
                selected_subset = temp_list
                
    return selected_subset

In [7]:
selected_models = ['ST4000DM000', 'ST8000DM002', 'ST12000NM0007', 'ST8000NM0055', 'ST3000DM001', 'ST4000DX000']

In [8]:
for disk_model_name in selected_models:

    hdd = pd.read_csv("../"+str(number_of_days)+'/'+str(disk_model_name)+'.csv', header=None)
    #hdd = pd.read_csv("../dataset_1.csv")
    hdd = hdd.drop(hdd.columns[6], axis=1)
    hdd = hdd.drop(hdd.columns[9], axis=1)
    hdd = hdd.drop(hdd.columns[14], axis=1)
    hdd = hdd.drop(hdd.columns[13], axis=1)
    hdd = hdd.dropna()
#    print(hdd.head())
    
    hdd_extra = pd.read_csv("../2019_files/"+str(number_of_days)+'/'+str(disk_model_name)+'.csv', header=None)
#    print(hdd_extra.head())
    hdd_extra = hdd_extra.drop(hdd_extra.columns[6], axis=1)
    hdd_extra = hdd_extra.drop(hdd_extra.columns[9], axis=1)
    hdd_extra = hdd_extra.drop(hdd_extra.columns[14], axis=1)
    hdd_extra = hdd_extra.drop(hdd_extra.columns[13], axis=1)
    hdd_extra = hdd_extra.dropna()
    
    hdd_merged = [hdd, hdd_extra]
    result = pd.concat(hdd_merged)
#    print(result.head())
    #result = result.dropna()
    
    x = result.iloc[:, :-1].values
    y = result.iloc[:, -1].values
    
    from imblearn.under_sampling import RandomUnderSampler
    rus = RandomUnderSampler()
    
    del hdd
    del hdd_extra
    del hdd_merged
    del result
    gc.collect()
    
    X_resampled, y_resampled = rus.fit_resample(x, y)
    print(Counter(y_resampled))
#    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)
    clf=RandomForestClassifier()
    clf.fit(X_resampled,y_resampled)

KeyboardInterrupt: 